<a href="https://colab.research.google.com/github/spaziochirale/CorsoPythonML/blob/master/ExampleCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.



In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Costruire una Convolutional Neural Network utilizzando gli Estimators



Il modulo `tf.layers` fornisce un'API di alto livello che semplifica la costruzione di Reti Neurali. 
In particolare mette a disposizione metodi che facilitano la creazioni di layers di tipo *dense*, cioè completamente connessi, e layer convoluzionali, permettendo l'aggiunta di funzioni di attivazioni specifiche e operazioni di *dropout*.
In questo tutorial,
viene spiegato come utilizzare `layers` per costruire una CNN da applicare al MNIST data set classico.

![cifre manoscritte 0–9 dal  MNIST data set](https://www.tensorflow.org/images/mnist_0-9.png)

Il [MNIST dataset](http://yann.lecun.com/exdb/mnist/) comprende 60,000
esempi di training e 10,000  esempi di test di cifre 0–9 scritte a mano,
formattate come matrici monocromatiche di 28x28-pixel.

## Iniziamo

Per prima cosa inseriamo le operazioni di import per i moduli TensorFlow:

In [0]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)

In questo notebook costruiremo la rete un passo alla volta. Il codice completo può essere
[scaricato qui](https://www.tensorflow.org/code/tensorflow/examples/tutorials/layers/cnn_mnist.py).

## Introduzione alle Convolutional Neural Networks

Le reti CNN rappresentano lo stato dell'arte nell'ambito dei modelli per la classificazione di immagini.
Una CNN applica una serie di filtri alla matrice di pixel d'origine per estrarre e apprendere le feature di più alto livello, che successivamente gli strati densi della rete utilizzeranno per effettuare la classificazione.

Una rete CNN ha i seguenti componenti:

*   **Convolutional layers**, che applicano il numero specifico di filtri all'immagine. Per ciascuna sotto-area, lo strato esegue un insieme di operazioni matematiche per produrre un singolo valore nella output feature map.
    Gli strati convoluzionali utilizzano tipicamente una
    [ReLU activation function](https://en.wikipedia.org/wiki/Rectifier_\(neural_networks\)) all'output per introdurre non linearità al modello.

*   **Pooling layers**, che
    [riducono la complessità dei dati](https://en.wikipedia.org/wiki/Convolutional_neural_network#Pooling_layer)
   estratti dai layer convoluzionali con l'obiettivo di ridurre le dimensioni della
    feature map e quindi del tempo di elaborazione. Un algoritmo di pooling molto usato è il max pooling, che estrae dalla feature map delle sotto-regioni    (ad esempio di 2x2-pixel), mantenendo solo il valore massimo e scartando tutti gli altri

*   **Dense (fully connected) layers**, che eseguono la classificazione delle
    features estratte dai layers convoluzionali e downsampled dai
    pooling layers. In un dense layer, ogni nodo in di un layer è connesso ad
    ogni nodo idel layer precedente.

Solitamente una CNN è composta da uno stack di moduli convoluzionali che eseguono l'estrazione delle features. Ogni modulo è costituito da un layer convoluzionale seguito da un pooling layer. L'ultimo modulo convoluzionale è seguito da uno o più strati di tipo dense che effettuano la classificazione.
L'ultimo strato dense della rete contiene tanti nodi quante sono le categorie che il modello è chiamato a prevedere. Tali nodi hanno una
[softmax](https://en.wikipedia.org/wiki/Softmax_function) activation function per generare valori nell'intervallo 0–1 per ciascun nodo (la somma di tutti i nodi softmax è pari ad 1). Il valore può essere interpretato come la confidenza stimata dalla rete che l'immagine appartenga alla specifica categoria rappresentata dal nodo.

Per un approfondimento si consulti [Convolutional Neural Networks for Visual Recognition course material](https://cs231n.github.io/convolutional-networks/).

## Costruiamo il Classifier CNN per il MNIST Data set

Nel nostro modello, utilizzeremo l'architettura seguente:

1.  **Convolutional Layer #1**: Applica 32 filtri 5x5  (per estrarre aree di 5x5-pixel), con la ReLU activation function
2.  **Pooling Layer #1**: Esegue il max pooling con un filtro 2x2 filter e passo (stride) di 2
    (le regioni non si sovrappongono)
3.  **Convolutional Layer #2**: Applica 64 filtri 5x5, con la ReLU activation
    function
4.  **Pooling Layer #2**: Identico al pooling layer #1
5.  **Dense Layer #1**: 1,024 neuron1, con dropout regularization rate di 0.4
    (probabilità di 0.4 che un dato elemento sarà eliminato durante il training)
6.  **Dense Layer #2 (Logits Layer)**: 10 neuron1, uno per ciascun digit (0–9).

Il modulo `tf.layers` contiene metodi per creare ciascuno dei layer elencati sopra:

*   `conv2d()`. Costruisce un layer convoluzionale bidimensionale. Riceve come parametri il numero di filtri, il kernel size di ciascun filtro, il padding e la funzione di attivazione.
*   `max_pooling2d()`. Costruisce un pooling layer bidimensionale che usa l'algoritmo di 
    max-pooling. Riceve come argomenti la dimensione dei filtri e lo stride.
*   `dense()`. Costruisce un dense layer. Riceve come argomenti il numero di neuroni e l'activation
    function.

Ciascuno di questi metodi accetta come input un tensore e restituisce un tensore trasformato come output.
In questo modo è semplice interconnettere i layer, utilizzando l'output del layer precedente come input del layer seguente.

Per realizzare un Estimator in TensorFlow, è necessario scrivere una funzione `cnn_model_fn` in modo conforme all'interfaccia specificata per le API TensorFlow's Estimator (si veda [Creare l'Estimator](#create-the-estimator)). 
Questa funzione riceve come argomenti le feature del data set MNIST, le relative labels, e il parametro *mode* (il cui valore assumerà uno degli stati indicati da `tf.estimator.ModeKeys`: `TRAIN`, `EVAL`, `PREDICT`);
Questa funzione configura la CNN; e restituisce le predictions, il valore della funzione loss, e il riferimento all'operazione di training.
Il codice della funzione è il seguente:

In [0]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

Nelle sezioni seguenti, in cui ciascun titolo corrisponde a ciascuno dei blocchi di codice scritti sopra, viene specificato in maggiore dettaglio il significato del codice utilizzato per definire ciascun layer e le modalità utilizzate per calcolare il loss, configurare l'operazione di training e generare le previsioni. 


### Input Layer

I metodi del modulo `layers` per creare gli strati convoluzionali e di pooling per i dati bidimensionali delle immagini richiedono che i tensori di input abbiano uno *shape* di
<code>[<em>batch_size</em>, <em>image_height</em>, <em>image_width</em>,
<em>channels</em>]</code> di default. Questo comportamento può essere modificato utilizzando il parametro
<code><em>data_format</em></code>; definito nel modo seguente:

*   `batch_size` —Dimensione del sottoinsieme di esempi da utilizzare quando si esegue il gradient descent durante la fase di training.
*   `image_height` —Altezza delle immagini di esempio.
*   `image_width` —Larghezza delle immagini di esempio.
*   `channels` —Numero dei canali di colore nelle immagini di esempio. Per immagini a colori, il numero di canali è 3 (red, green, blue). Per immagini monocromatiche il numero di canali è soltanto 1 (black).
*   `data_format` —Una stringa di valore `channels_last` (default) oppure `channels_first`.
      `channels_last` corrisponde ad inputs con shape
      `(batch, ..., channels)` mentre `channels_first` corrisponde a
      inputs con shape `(batch, channels, ...)`.

Nel nostro caso, il dataset MNIST è composto da immagini monocromatiche di 28x28 pixel, pertanto lo shape atteso per il nostro input layer è <code>[<em>batch_size</em>, 28, 28,
1]</code>.

Per convertire la nostra feature map (`features`) di input a questo shape, possiamo eseguire la seguente operazione `reshape`:

```
input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
```

Si noti che abbiamo indicato `-1` per il batch size. Questa notazione indica che tale dimensione debba essere calcolata dinamicamente in base al numero di valori di input in
`features["x"]`, mantenendo costanti tutte le altre dimensioni. Questo permette di trattare `batch_size` come un iperparametro utilizzabile per effettuare il tuning. 
Ad esempio, se alimentiamo la rete in batch di 5, `features["x"]` conterrà
3,920 valori (un valore per ciascun pixel in ciascuna immagine), e `input_layer` avrà uno shape di `[5, 28, 28, 1]`. Analogamente, se forniamo esempi in batch di
100, `features["x"]` conterrà 78,400 valori, e `input_layer` avrà uno
shape di `[100, 28, 28, 1]`.

### Convolutional Layer #1

Nel nostro primo strato convoluzionale vogliamo applicare all'input 32 filtri 5X5, seguiti da una funzione di attivazione ReLU. Possiamo utilizzare il metodo `conv2d()` del modulo
`layers` in questo modo:

```
conv1 = tf.layers.conv2d(
    inputs=input_layer,
    filters=32,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)
```

Il parametro `inputs` specifica il tensore di input, che deve avere shape di
<code>[<em>batch_size</em>, <em>image_height</em>, <em>image_width</em>,
<em>channels</em>]</code>. Connettiamo il primo convolutional layer
all' `input_layer`, che ha shape <code>[<em>batch_size</em>, 28, 28,
1]</code>.

Nota: `conv2d()` al contrario, si aspetta uno shape di `[<em>batch_size</em>, <em>channels</em>, <em>image_height</em>, <em>image_width</em>]` quando viene indicato il parametro `data_format=channels_first`.

Il parametro `filters` specifica il numero di filtri da applicare (nel nostro caso 32), e
`kernel_size` specifica la dimensione dei filtri come `[<em>height</em>,
<em>width</em>]</code> (nel nostro caso <code>[5, 5]`).

<p class="tip"><b>TIP:</b> Nel caso in cui filter height e width abbiano lo stesso valore è possibile specificare un
singolo intero per <code>kernel_size</code>—ad esempio <code>kernel_size=5</code>.</p>

Il parametro `padding` specifica uno dei seguenti valori
(case-insensitive): `valid` (default) oppure `same`. 
Nel nostro caso abbiamo indicato `padding=same` per specificare che il tensore di output deve avere le stesse dimensioni del tensore di input,
TensorFlow aggiunge zeri agli estremi del tensore di input per mantenere height e width di 28. (Senza padding,
una convoluzione 5x5 su un tensore 28x28 produrrebbe un tensore 24x24).

Il parametro `activation` specifica la funzione di attivazione da applicare all'output della convoluzione. 
Nel nostro caso abbiamo specificato una attivazione ReLU utilizzando il valore
`tf.nn.relu`.

Il tensore di output prodotto nel nostro caso da `conv2d()` ha uno shape di
<code>[<em>batch_size</em>, 28, 28, 32]</code>:  Le dimensioni height e width
hanno gli stessi valori dell'input, ma con 32 canali che mantengono gli output da ciascuno dei filtri.

### Pooling Layer #1

A questo punto possiamo connettere il nostro primo strato di pooling al layer convoluzionale che abbiamo appena creato.
Possiamo utilizzare il metodo `max_pooling2d()` del modulo `layers` per costruire uno strato che effettua il max pooling con un filtro 2x2 e uno stride di 2:

```
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
```

Anche in questo caso, `inputs` specifica il tensore di input che ha uno shape di
<code>[<em>batch_size</em>, <em>image_height</em>, <em>image_width</em>,
<em>channels</em>]</code>. Nel nostro caso, il tensore di input è `conv1`, cioè l'output del primo strato convoluzionale, che ha uno shape di <code>[<em>batch_size</em>,
28, 28, 32]</code>.

Nota: Così come <code>conv2d()</code>, anche <code>max_pooling2d()</code> si aspetterà uno shape di <code>[<em>batch_size</em>, <em>channels</em>, 
<em>image_height</em>, <em>image_width</em>]</code> se viene utilizzato il parametro
<code>data_format=channels_first</code>.

Il parametro `pool_size` specifica la dimensione del filtro di max pooling pari a
<code>[<em>height</em>, <em>width</em>]</code> (nel nostro caso `[2, 2]`). Se entrambe le dimensioni hanno lo stesso valore è possibile specificare un unico numero intero (ad esempio
`pool_size=2`).

Il parametro `strides` specifica la dimensione del passo. Nel nostro caso abbiamo impostato uno stride
di 2, che indica che le aree estratte dal filtro devono essere separate da 2 pixel sia in altezza che in larghezza (per un filtro 2x2 questo comporta che nessuna regione si sovrappone alle altre). È possibile impostare passi differenti per altezza e larghezza specificando una tupla o una lista (ad esempio `stride=[3, 6]`).

Il nostro tensore di output prodotto da `max_pooling2d()` (`pool1`) ha uno shape di
<code>[<em>batch_size</em>, 14, 14, 32]</code>: il filtro 2x2 riduce le dimensioni height e width, ciascuna del 50%.

### Convolutional Layer #2 e Pooling Layer #2

Possiamo quindi connettere un secondo strato convoluzionale seguito da un secondo strato di pooling nello stesso modo utilizzando 
`conv2d()` e `max_pooling2d()` .
Per il secondo layer convoluzionale abbiamo configurato 64 5x5 filtri con funzione di attivazioneReLU, e per il pooling layer #2 abbiamo utilizzato le stesse specifiche del pooling layer #1 (un filtro max pooling 2x2 con stride di 2):

```
conv2 = tf.layers.conv2d(
    inputs=pool1,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
```

Si noti che il convolutional layer #2 riceve come input il tensore di output del primo pooling
layer (`pool1`) e produce in output il tensore `conv2`. `conv2`
ha uno shape di <code>[<em>batch_size</em>, 14, 14, 64]</code>, stesse dimensioni in altezza e larghezza di `pool1` (in conseguenza del parametro `padding="same"`), e 64 canali a seguito dei 64
filtri che sono stati applicati.

Pooling layer #2 riceve `conv2` come input, fornendo `pool2` come output. `pool2`
ha uno shape di <code>[<em>batch_size</em>, 7, 7, 64]</code> (riduzione del 50% per height e width dai valori di `conv2`).

### Dense Layer

Dopo i due stadi convoluzionali + pooling, aggiungiamo uno strato denso con 1.024 neuroni funzione di attivazione ReLU.
Questa sezione effettuerà la classificazione delle features estratte dalla parte convoluzionale.
Prima di connettere lo strato denso è necessario "appiattire" la feature map output di `pool2`, che è in forma bidimensionale
con shape <code>[<em>batch_size</em>,
<em>features</em>]</code>, ottenendo un tensore di una dimensione + il batch size:

```
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
```

Nell'operazione `reshape()`  `-1` implica che la dimensione *`batch_size`*
sarà calcolata dinamicamente in base alla dimensione dell'input. Ogni campione ha 7 (`pool2` height) * 7 (`pool2` width) * 64
(`pool2` channels) feature, e noi vogliamo per  `features` una dimensione di  7 * 7 * 64 (3136 in totale). Il tensore di output, `pool2_flat`, ha shape
<code>[<em>batch_size</em>, 3136]</code>.

Possiamo quindi utilizzare il metodo `dense()` di `layers` per connettere lo strato di classificazione:

```
dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
```

Il parametro `inputs` specifica il tensore di input cioè la nostra feature map dopo l'operazione di flattening,
`pool2_flat`. Il parametro `units` specifica il numero di neuroni del dense
layer (1.024). Il parametro `activation` specifica la activation function; anche in questo caso,
abbiamo usato `tf.nn.relu` per aggiungere una funzione ReLU.

Allo scopo di prevenire fenomeni di overfitting e ottimizzare le prestazioni del modello, abbiamo deciso di utilizzare la strategia di dropout regularization
applicata al dense layer, attraverso la chiamata al metodo `dropout` del modulo `layers`:

```
dropout = tf.layers.dropout(
    inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
```

Come sempre, `inputs` indica il tensore di input, che è il tensore di output del
dense layer (`dense`).

Il parametro `rate` specifica il grado di dropout; in questo caso  `0.4`, che significa che il
40% degli elementi saranno eliminati durante il training.

Il parametro `training` è un valore booleano che specifica se il modello è chiamato o meno in modalità di training; il dropout viene eseguito solo se il valore di
`training` è `True`. All'interno della chiamata, viene quindi verificato se il valore di `mode` passato alla funzione
`cnn_model_fn` è `TRAIN`.

Il tensore di output `dropout` ha shape <code>[<em>batch_size</em>, 1024]</code>.

### Logits Layer

L'ultimo layer della nostra rete e un layer di tipo logits. Questo strato restituisce i valori complessivi della previsione effettuata dalla rete.
Viene creato un dense layer con 10 neuron1 (uno per
ciascuna classe target 0–9), con funzione di attivazione lineare (il default):

```
logits = tf.layers.dense(inputs=dropout, units=10)
```

L'output finale della CNN, `logits`, ha shape `[batch_size, 10]`.

### Generare le Previsioni {#generate_predictions}

Il logits layer del nostro modello restituisce le previsioni in forma di valori raw  in un tensore di shape
<code>[<em>batch_size</em>, 10]</code>. 
Convertiamo questi valori
raw in due diversi formati che possano essere restituiti dalla model function:

*   La **predicted class** per ciascun campione: una cifra nel range 0–9.
*   Le **probabilities** per ciascuna probabile categoria di ciascun campione: la probabilità che il campione rappresenti uno 0, un 1, un 2, etc.

Dato un campione, la previsione della rete è indicata dalla cifra con la maggiore probabilità. possiamo calcolarla con il metodo `tf.argmax`:

```
tf.argmax(input=logits, axis=1)
```

Il parametro `input` specifica il tensore dal quale estrarre i valori massimi —nel nostro caso `logits`. Il parametro `axis` specifica gli assi del tensore `input`
lungo il quale trovare il valore massimo. Nel nostro caso, vogliamo trovare il valore più grande lungo la dimensione con indice 1, che corrisponde alle nostre previsioni.
(Si ricordi che il tensore logits ha shape <code>[<em>batch_size</em>,
10]</code>).

Possiamo derivare le probabilità dal logits layer applicando la funzione di attivazione softmax utilizzando `tf.nn.softmax`:

```
tf.nn.softmax(logits, name="softmax_tensor")
```

Nota: Abbiamo usato il parametro `name` per battezzare in modo esplicito questa operazione `softmax_tensor`, in questo modo sarà possibile referenziarla quando imposteremo il logging per i valori softmax nella sezione ["Impostare un Logging Hook"](#set-up-a-logging-hook)).

Le previsioni vengono quindi compilate in un dizionario e restituite tramite un oggetto `EstimatorSpec`:

```
predictions = {
    "classes": tf.argmax(input=logits, axis=1),
    "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
}
if mode == tf.estimator.ModeKeys.PREDICT:
  return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
```

### Calcolo del Loss {#calculating-loss}

Sia nella fase di training che nella fase di valutazione della bontà del modello, è necessario definire una
[loss function](https://en.wikipedia.org/wiki/Loss_function)
che misuri quanto accurate siano le prevsioni del modello in relazione alle categorie target.
Per problemi di classificazione come il MNIST,
[cross entropy](https://en.wikipedia.org/wiki/Cross_entropy) è la metrica di loss usata nella maggior parte dei casi. 
il codice seguente calcola la funzione cross entropy quando la rete è eseguita in modalità `TRAIN` o `EVAL`:

```
loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
```

il tensore `labels` contiene una lista di indici di previsione per il campione passato, ad esempio. `[1,
9, ...]`. `logits` contiene gli output lineari dell'ultimo layer. 

`tf.losses.sparse_softmax_cross_entropy`, calcola la softmax crossentropy
da questi due input in modo efficiente.

### Configurare l'operazione di Training

Configuriamo ora il nostro modello in modo da ottimizzare il valore del loss calcolato nel modo indicato in precedenza, durante il training. Useremo un parametro di learning rate pari a 0.001 e il metodo
[stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)
come algoritmo di ottimizzazione:

```
if mode == tf.estimator.ModeKeys.TRAIN:
  optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
  train_op = optimizer.minimize(
      loss=loss,
      global_step=tf.train.get_global_step())
  return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
```

Nota: Per i dettagli su come configurare una operazione di training per le model function degli  Estimator si veda ["Defining the training op for the model"](../../guide/custom_estimators.md#defining-the-training-op-for-the-model) nel tutorial ["Creating Estimations in tf.estimator"](../../guide/custom_estimators.md).

### Aggiungere le metriche di valutazione

Per definire la metrica accuracy nel nostro modello abbiamo definito il dizionario `eval_metric_ops`  quando la funzione è chiamata in EVAL
mode nel modo seguente:

```
eval_metric_ops = {
    "accuracy": tf.metrics.accuracy(
        labels=labels, predictions=predictions["classes"])
}
return tf.estimator.EstimatorSpec(
    mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
```

<a id="train_eval_mnist"></a>
## Training e Valutazione del Classificatore CNN MNIST

Una volta scritta la model function del modello MNIST CNN siamo pronti per addestrare e testare la rete.

### Caricare i dati di Training e Test

Il codice seguente carica i due dataset di training e test:

In [0]:
# Load training and eval data
((train_data, train_labels),
 (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

I dati di training delle feature (i valori raw dei pixel per le 55.000 immagini di cifre scritte a mano) e le relative label (valori 0–9 corrispondenti a ciascuna immagine) sono memorizzati nei due vettori di tipo [numpy
arrays](https://docs.scipy.org/doc/numpy/reference/generated/numpy.array.html)
 `train_data` e `train_labels`. 
Analogamente i dati di test (10.000 immagini) e le label relative sono memorizzati in `eval_data`
e `eval_labels`.

### Creare l'Estimator {#create-the-estimator}

A questo punto possiamo creare un `Estimator` (una classe TensorFlow per effettuare ad alto livello il training, la valutazione del modello e le previsioni) per il nostro modello:

In [0]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

Il parametro `model_fn` specifica la model function da utilizzare per le fasi di training,
valutazione, e previsione; abbiamo valorizzato il parametro con la `cnn_model_fn` creata nella sezione
["Costruire il Classificatore CNN per il dataset MNIST"](#building-the-cnn-mnist-classifier) Il parametro
`model_dir` specifica la directory dove i dati del modello (checkpoints) saranno salvati (nel nostro esempio abbiamo usato `/tmp/mnist_convnet_model`, ma il lettore può scegliere una qualunque altra destinazione sul proprio file system).

Nota: Per maggiori dettagli sulle API TensorFlow `Estimator` si rimanda al tutorial [Creating Estimators in tf.estimator](../../guide/custom_estimators.md).

### Impostare un Logging Hook {#set_up_a_logging_hook}

Poiché la fase di addestramento di una CNN può essere molto lunga, è opportuno impostare una attività di logging per tracciare l'avanzamento e i progressi effettuati durante il training.
Possiamo utilizzare la classe TensorFlow `tf.train.SessionRunHook` per creare un
`tf.train.LoggingTensorHook`
che effettuerà il log dei valori di probabilità dal softmax layer della nostra CNN:

In [0]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

Memorizziamo il dizionario dei tensori che intendiamo loggare in `tensors_to_log`. Ciascuna key rappresenta la label di nostra scelta che sarà riportata nell'output del log, e la label corrispondente è il nome di un `Tensor` nel grafo TensorFlow. Nel nostro caso, le
`probabilities` possono essere trovate nel `softmax_tensor`, il nome che abbiamo attribuito in precedenza all'operazione softmax
quando abbiamo generato le probabilità in `cnn_model_fn`.

Nota: se non viene assegnato un nome esplicito tramite il parametro `name`, TensorFlow assegnerà un nome di default all'operazione. Per scoprire il nome assegnato all'operazione si può visualizzare il grafo utilizzando la [TensorBoard](../../guide/graph_viz.md)) o abilitare il [TensorFlow Debugger (tfdbg)](../../guide/debugger.md).

A questo punto creiamo `LoggingTensorHook`, passando `tensors_to_log` al parametro
`tensors`. Abbiamo impostato `every_n_iter=50`, per specificare che le probabilità devono essere loggate ogni 50 step.

### Training del Modello

Per addestrare il modello occorre definire una `train_input_fn`
e richiamare il metodo `train()` sull'oggetto `mnist_classifier`. 
Nella chiamata `numpy_input_fn` passiamo le feature e le label dei dati di training rispettivamente come
`x` (dizionario) e `y`. Impostiamo un `batch_size` di `100`.
`num_epochs=None` indica che il modello svolgerà il training finché il numero specificato di step non sarà raggiunto. Abbiamo anche impostato `shuffle=True` per mescolare casualmente i dati di training. 
A questo punto possiamo lanciare uno step di training ed effettuare il log dell'output:

In [0]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=1,
    hooks=[logging_hook])

A questo punto, evitando di effettuare il log di ciascuno step, impostiamo `steps=1000` per effettuare un training più lungo, ma ancora per un tempo adatto a questo esempio. 
Il training di reti CNN è molto costoso in termini computazionali.  20.000 step è un valore che potrebbe fornire una accuratezza maggiore. 

In [0]:
mnist_classifier.train(input_fn=train_input_fn, steps=1000)

### Valutare il Modello

Una volta effettuato il training possiamo richiamare il metodo `evaluate` per valutare l'accuratezza raggiunta sui dati di test utilizzando le metriche definite nel parametro `eval_metric_ops` della `model_fn`.


In [0]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

Per definire `eval_input_fn`, abbiamo impostato `num_epochs=1`, in modo che il modello effettui una iterazione sul set di dati e restituisca i risultati. 
`shuffle=False` imposta l'iterazione in modo sequenziale sui dati.

## Approfondimenti

Per maggiori dettagli sui TensorFlow Estimators e le CNNs in TensorFlow, si consiglia:

*   [Creating Estimators in tf.estimator](../../guide/custom_estimators.md)
    Introduzione a TensorFlow Estimator API. Configurare Estimator, scrivere una model function, calcolare loss, e
    definire una training op.
*   [Advanced Convolutional Neural Networks](../../tutorials/images/deep_cnn.md) Costruire un MNIST CNN classification model
    *senza estimator* utilizzando API TensorFlow di basso livello.